In [75]:
#save degree of rotation into CSV
    # construct dataframe : columnA filename, columnB: section#, columnC: centroid of section, columnD: degree of rotation

#read degree of rotation
#read H&E image
#mask each section
    #read_DL_mask
    #label_DL_mask
    #label == numsec
#rotate H&E image
#crop H&E image

In [76]:
#this is crop_align HE and DL

In [77]:
# imcrop_masked = np.multiply(np.array(imcrop),objmask[..., None].astype(np.bool))

In [78]:
import pandas as pd
from PIL import Image
Image.MAX_IMAGE_PIXELS=None
import numpy as np
from skimage import util
from skimage.measure import label
from skimage.morphology import closing, square, remove_small_objects, remove_small_holes
from skimage.transform import rotate
from math import atan2, degrees
import glob,os
from natsort import natsorted
from copy import deepcopy
import cv2
from time import time
from skimage.measure import regionprops
from matplotlib import pyplot as plt

In [79]:
def rotate_image_cv2(mat, angle):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat

In [80]:
# Define minimum sizes of tissue objects at 1um resolution
minTA = 60000
minTAhole = 100
minDermhole = 5000
minepisize=1000
whitespace=12 #define background pixel value

In [81]:
src = r"\\fatherserverdw\kyuex\clue images\1um"
dst = os.path.join(src, 'classification_v9_combined\crop_TA\crop_HE_DL')

In [82]:
degrot_LUT = pd.read_csv(r"\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\crop_TA\tmp\CLUEdegrot.csv")

In [83]:
old_id = -1

In [ ]:
#TODO: iterate through image instead of dataframes,so it's faster
for idx, LUT in degrot_LUT.iterrows():
    imname = LUT['imname']
    imID = LUT['imID']
    #if it's the same image but different sections, skip
    if(imID == old_id): continue

    degrot =  LUT['degrot']
    sec = LUT['secN']


    im_path =  r"\\fatherserverdw\kyuex\clue images\1um\{}.tif".format(imname)
    mask_path = r"\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\{}.tif".format(imname)
    im = Image.open(im_path)
    mask = Image.open(mask_path)
    TAbig = np.array(mask)


    #label mask
    (width, height) = (mask.width // 10, mask.height // 10) #downsize by 10
    mask_resized = mask.resize((width, height), resample=0) #nearest interpolation to rescale
    TA = np.array(mask_resized) # convert DL mask to numpy array
    # separate touching tissue sections using WATERSHED segmentation
    sure_foreground = (2 < TA) & (TA < whitespace - 1) #define tissue section conservatively by removing corneum==1, and removing fat==12, so this foreground does not touch adjacent tissue section for sure.

    sure_fg = closing(sure_foreground, square(3))  # 13sec
    # remove small objects not connected to the tissue section
    sure_fg = remove_small_objects(sure_fg, min_size=minTA, connectivity=2)  # 6sec

    sure_fg = remove_small_holes(sure_fg, area_threshold=minTAhole).astype(np.uint8)

    bw = closing(TA < whitespace, square(3))  # 12 is background
    bw = remove_small_objects(bw, min_size=minTA, connectivity=2)
    bw = remove_small_holes(bw, area_threshold=minTAhole)
    # morphological opening : erosion followed by an dilation to remove small objects
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(bw.astype(np.uint8), cv2.MORPH_OPEN, kernel, iterations=2)  # 2sec
    # dilate to be more lenient in defining tissue section area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    unknown = cv2.subtract(sure_bg, sure_fg).astype(np.bool)

    sure_fg_label = label(sure_fg).astype(np.int32)
    sure_fg_label = sure_fg_label + 1 #objects are 2~N
    sure_fg_label[unknown] = 0 #middle ground is 0

    #get labeled image
    TAbgr = cv2.cvtColor(TA, cv2.COLOR_GRAY2BGR) #convert downsized DL mask to BGR color channel
    label_image = cv2.watershed(TAbgr, sure_fg_label) #apply watershed using fg,bg,mg, defined mask

    # iterate each section
    epi = (TA == 1) | (TA == 2)
    derm = (2 < TA) & (TA < whitespace)
    derm = remove_small_holes(derm, area_threshold=minDermhole)

    epi2 = epi & ~derm
    epi2 = remove_small_objects(epi2, min_size=minepisize, connectivity=2)

    numsecmax = np.max(label_image)

    #TODO: for each section of the same image, you need to reopen the image each time, which takes up time
    for num, numsec in enumerate(range(1,numsecmax)):
        #this step updates the degrot of each section
        if num > 0:
            row = degrot_LUT.iloc[idx+num]
            degrot = row['degrot']

        print('section N: ', numsec, '/', numsecmax-1)
        msktmp = label_image == numsec+1
        stats = regionprops(msktmp.astype(np.uint8))
        centroid = stats[0].centroid #NOTE:this returns (y,x)!!
        cent = (10*centroid[1],10*centroid[0])

        TAtmp = deepcopy(TAbig)
        mskbig = cv2.resize(msktmp.astype(np.uint8), TAtmp.shape[::-1], interpolation=cv2.INTER_NEAREST)
        # delete non-ROI from scaled up ROI
        kernel = np.ones((20, 20), np.uint8)
        mskbig = cv2.dilate(mskbig, kernel, iterations=3)
        TAtmp[mskbig == 0] = 0  #1sec

        imcrop = np.multiply(np.array(im),TAtmp[..., None].astype('bool'))

        #crop it, rotate it, and then crop it again (or else the matrix is too big for cv2 to process)
        [xt, yt] = np.where(imcrop[:,:,0]) #tissue mask to point clouds,don't know why there are three arrays
        imrot = imcrop[np.min(xt):np.max(xt), np.min(yt):np.max(yt)]
        imrot2 = rotate_image_cv2(imrot, degrot)

        [xt, yt] = np.where(imrot2[:,:,0]) #tissue mask to point clouds,don't know why there are three arrays
        imrot3 = imrot2[np.min(xt):np.max(xt), np.min(yt):np.max(yt)]


        #repeat the step above for the mask
        [xt, yt] = np.where(TAtmp[:,:])
        mskrot =TAtmp[np.min(xt):np.max(xt), np.min(yt):np.max(yt)]
        mskrot2 = rotate_image_cv2(mskrot, degrot)

        [xt, yt] = np.where(mskrot2)
        mskrot3 = mskrot2[np.min(xt):np.max(xt), np.min(yt):np.max(yt)]

        # if dermis is above epidermis, flip it
        [xt, yt] = np.where(mskrot3)
        [xt2, yt2] = np.where((mskrot3 == 1) | (mskrot3 == 2))
        if np.mean(xt) - np.mean(xt2) < 0:
            imrot3 = np.rot90(np.rot90(imrot3))
            mskrot3 = np.rot90(np.rot90(mskrot3))

        imrot3[imrot3 == 0] = 235  # assign whitespace value to background
        mskrot3[mskrot3 == 0] = whitespace

        Image.fromarray(imrot3).save(
            os.path.join(*[dst, 'im','{}_sec{:02d}.png'.format(imname, numsec)]))

        Image.fromarray(mskrot3).save(
            os.path.join(*[dst, 'mask','{}_sec{:02d}.png'.format(imname, numsec)]))

    old_id = imID